In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from unisim import ExactUniSim, ApproxUniSim

Using onnx with cpu


/Users/elieb/git/unisim/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# usim = ExactUniSim(store_data=True, use_tf_knn=False)
usim = ExactUniSim(store_data=True, 
                   use_tf_knn=False)
#usim = ApproxUniSim(store_data=True)
usim.info()


UniSim is storing a copy of the indexed data
if you are using large data corpus consider disable this behavior using store_data=False
[Embedder]
|-batch_size:128
[Indexer]
|-is_exact:True
|-use_tf_knn:False
|-store index data:True


In [13]:
apa = 'apa' * 1024
g, e = usim.text.embed('2 strings one family')
print(g.shape, len(e))
g, e = usim.text.batch_embed(['2 strings one family', apa])
print(g.shape, len(e))

0it [00:00, ?it/s]


(256,) 1


0it [00:00, ?it/s]

(2, 256) 2


In [15]:
N = 100
ts = time()
for _ in range(N):
    usim.text.embed('2 strings one family')
total = (time() - ts) * 1000
print(f'total ms {round(total)} - {round(total/N)}ms')

Computing partial embeddings: 100%|██████████| 1/1 [00:00<00:00, 134.20embeddings/s]

total ms 462 - 5ms


In [28]:
t = 'apa' * 1024
d = usim.text.similarity('apapa', t)
print('long range', d.is_global_match, d.is_partial_match, d.distance)


# golden test 0.48
d = usim.text.similarity('Doubting dreams'.lower(), 'rough winds do shake the darling buds of may,'.lower())
print('golden', d.distance)

d = usim.text.similarity('I love icecreams', 'I love icecreams')
print('exact', d.distance)


Computing partial embeddings: 100%|██████████| 7/7 [00:00<00:00, 216.43embeddings/s]


long range False False 0.2628413438796997


Computing partial embeddings: 100%|██████████| 2/2 [00:00<00:00, 198.14embeddings/s]


golden 0.48702412843704224


Computing partial embeddings: 100%|██████████| 2/2 [00:00<00:00, 276.92embeddings/s]

exact 0.9999998211860657


In [25]:
example = 't' * 4096
print(len(example))


4096


In [55]:
usim.text.reset_index()
ts  = time()
chunk = 'cookies'
partial =  'k' * 530 + chunk
usim.text.batch_index([example,  partial,  'I love icecreams', 'tea oh tea', 'cookiess!!!', 'no cookies for you'])
usim.text.batch_index(['icecream or icecream sandwich that is the question'])


rc = usim.text.batch_search(['I love icecreams', 'my icecream', 'cookiess!!!', chunk],
                           pk=2, gk=2)
total = (time() - ts) * 1000
print(f'total ms {round(total)}')
print(rc)
usim.viz.result(rc.results[3])

Computing partial embeddings: 100%|██████████| 4/4 [00:00<00:00, 331.79embeddings/s]

total ms 68
Partial Match: 4
Query 3: "cookies"
  idx  is_global    global_sim    is_partial    partial_sim    content
-----  -----------  ------------  ------------  -------------  ------------------------------
    4  True         0.92          True          0.92           cookiess!!!
    5  False        0.8           False                        no cookies for you
    1  False                      True          0.93           kkkkkkkkkkkkkkkkkkkkkkkkkkkkkk


In [49]:
usim.text.indexer.info()

[Indexer info]
|-is_exact: True
|-use_tf_knn: False
